In [53]:
import pandas as pd
import re
import numpy
import os
import math
from os import listdir
from os.path import isfile, join
from sklearn.svm import SVR
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn import svm
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

In [54]:
def formatTweets(fileLocation):
    dataFrameVals = pd.read_excel(fileLocation)
    p = SnowballStemmer("english")
    stopWords = set(stopwords.words("english"))
    tweetList=dataFrameVals['Tweet']
    classList=dataFrameVals['Class']
    formattedTweets = []
    classes = []
    for record,label in zip(tweetList,classList):
        if(isinstance(label, str) != True):
            if (label !=2 and math.isnan(label) !=True):
                regexHTMLRemove = re.compile(r'<.*?>')
                tempString = regexHTMLRemove.sub('',str(record)).lower()             
                #Replace all URLS
                tempString = re.sub(r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',' ',tempString)
                #get all A to Z
                tempString = re.sub(r"[^A-Za-z ]+", ' ', tempString) 
                listOfWords = tempString.split()
                newList = []
                for word in listOfWords:
                    if word not in stopWords:
                        newList.append(p.stem(word))
                tempString = " ".join(str(x) for x in newList)              
                formattedTweets.append(tempString)
                classes.append(label)
    return formattedTweets,classes

In [55]:
tweets,classlabels=formatTweets(r"C:\Users\Shvetha\Desktop\Data Mining\Project\Project 2\Data\JOIN.xlsx");

In [57]:
#tf idf matrix for features 
train_set=tweets
vectorizer=CountVectorizer()
train_matrix=vectorizer.fit_transform(train_set)
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(train_matrix)
tf_idf_train= tfidf.transform(train_matrix)

In [58]:
clf = svm.LinearSVC()
for train,test in kf.split(tf_idf_train):
    predicted=[]
    truevalues=[]
    trainlabels=[]
    temp1=[]
    temp2=[]
    sumerror=0
    for i in train:
        temp1.append(tf_idf_train[i].toarray()[0])
        trainlabels.append(classlabels[i])
    trainset=sparse.csc_matrix(numpy.asarray(temp1))
    for i in test:
        temp2.append(tf_idf_train[i].toarray()[0])
        truevalues.append(classlabels[i])
    testset=sparse.csc_matrix(numpy.asarray(temp2))
    clf.fit(trainset,trainlabels)
    predicted=clf.predict(testset)
    for i in range(len(predicted)):
        if(predicted[i]!=truevalues[i]):
            sumerror=sumerror+1
    avgerror=avgerror+float(sumerror)/len(test)
    sumprec_1=sumprec_1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[0][0]
    sumprec1=sumprec1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[0][1]
    sumrec_1=sumrec_1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[1][0]
    sumrec1=sumrec1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[1][1]
    sumf_1=sumf_1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[2][0]
    sumf1=sumf1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[2][1]
print("avg prec 1",float(sumprec1)/10)
print("avg recall 1",float(sumrec1)/10)
print("avg prec -1",float(sumprec_1)/10)
print("avg recall -1",float(sumrec_1)/10)
print("avg fsc 1",float(sumf1)/10)
print("avg fs -1",float(sumf_1)/10)
print("avg error",float(avgerror)/10)

avg prec 1 0.42366670782853005
avg recall 1 0.4914166305229923
avg prec -1 0.6584931117095305
avg recall -1 0.5845103153381425
avg fsc 1 0.45128388447848494
avg fs -1 0.6119691748734568
avg error 0.46379865504536066
